# **PSI Model Synthesis Notebook for 14 Feb 2022 (CR 2254)**

Tom Schad 

Model runs at :  https://www.predsci.com/data/runs/

For reference, Del Zanna & DeLuca (1018)
Hot AR sim is multiT model.  T=6,6,2.6.45,6.6.  Log EM of 26.8,27.2,28.2,27.9
constant density of 1e9 cm^-3

    photons cm^-2 s^-1 arcsec^-2 
    10747 -->  430, 240,35
    10798 -->  340, 140, 21
    14305 --> 262(170), 51 (130), 37
    39343 --> 10, 29, 6

In [251]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi
import sunpy.coordinates.sun as sun
import sunpy.sun.constants as constants
plt.rcParams["image.origin"] = 'lower'   ## have to worry about flips? 
plt.rcParams["image.interpolation"]='nearest'
from scipy.interpolate import RegularGridInterpolator as rgi
from scipy.interpolate import interp1d
import multiprocessing
import tqdm
from matplotlib.colors import LogNorm
import allenSolRad as allen
from scipy.io import readsav
import sunpy.map

def euler_ry(alpha):
    '''Euler rotation matrix about y axis '''
    ry = np.array([[ np.cos(alpha), 0., np.sin(alpha)],
                   [            0., 1., 0.],
                   [-np.sin(alpha), 0., np.cos(alpha)]])
    return ry

def euler_rz(alpha):
    '''Euler rotation matrix about z axis '''
    rz = np.array([[ np.cos(alpha), -np.sin(alpha), 0.],
                   [ np.sin(alpha),  np.cos(alpha), 0.],
                   [            0.,             0., 1.]])
    return rz

## **Create an instance of the psi.Model class to load and interact with PSI coronal model data**

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration. 
- I had selected 12 UTC for the download time.  I'm not sure at this time where timestep information in the PSI models made be located.

In [2]:
## https://www.predsci.com/data/runs/cr2194-medium/hmi_mast_mas_std_0101/
## carrington rotation 2194
modelName = 'cr2254_high___hmi_mast_mas_std_0101'
corona = psi.Model('/data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/corona/')

In [3]:
## return the instance name for some basic information on the class
corona

psi Model class
    ---------------------
    Data Directory Names: /data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/corona/
    Number of longitude samples: 299
    Number of latitude samples: 142
    Number of radial samples: 254
    Data shape: (299, 142, 254)
    
    Variables: 
    lons -- Longitudes [rad]
    lats -- Latitudes [rad]
    rs   -- Radial samples [solar radii units]
    temp -- temperature [K]
    ne -- electron density [cm^-3]
    bx,by,bz  -- Cartesian components of magnetic field [G]
    vx,vy,vz  -- Cartesian components of velocity field [km/s]
    bmag      -- total magnetic field intensity [B]
    thetaBlocal -- location inclination of magnetic field in solar frame [rad]
    

In [4]:
kk = 25

fig,ax = plt.subplots(2,2,figsize = (8,5))
ax = ax.flatten()
im0 = ax[0].imshow(corona.bmag[:,:,kk].T*np.cos(corona.thetaBlocal[:,:,kk]).T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.thetaBlocal[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[2].imshow(corona.temp[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[3].imshow(corona.ne[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
labels = 'Magnetogram','Local B Inclination','Temperature','Electron Density'
for n in range(4): ax[n].set_title(labels[n])

im0.set_clim(-25,25)

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)
 axi.set_ylim(np.pi,0)

fig.suptitle(modelName + '\nRadial Coordinate: ' + str(corona.rs[kk]))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is the GONG synoptic map 

<img src="https://gong.nso.edu/data/magmap/QR/mqj/202202/mrmqj220221/mrmqj220221t2004c2254_000.jpg" />

In [5]:
fig,ax = plt.subplots(3,1,figsize = (5,6),sharex=True) 
ax = ax.flatten()
ax[0].plot(corona.rs,np.mean(corona.ne,axis=(0,1)),'.-',label = 'Mean Ne',markersize=1, lw=0.5) 
ax[0].set_yscale('log')
ax[1].plot(corona.rs,np.mean(corona.temp,axis=(0,1)),'g.-',label = 'Mean Temp',markersize=1, lw=0.5) 
ax[1].set_yscale('log')
ax[2].plot(corona.rs,np.mean(corona.bmag,axis=(0,1)),'k.-',label = 'Mean B',markersize = 1, lw=0.5) 
ax[2].set_yscale('log')
for axi in ax: 
    axi.set_xscale('log')
    axi.set_yscale('log')
    axi.legend(loc = 'upper right')
ax[1].set_yscale('linear')
ax[-1].set_xlabel('Solar Radii')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Solar Radii')

In [6]:
fig,ax = plt.subplots(3,1,figsize = (5,6),sharex=True) 
ax = ax.flatten()
ax[0].plot((corona.rs-1)*constants.radius.value/1e6,np.mean(corona.ne,axis=(0,1)),'.-',label = 'Mean Ne',markersize=1, lw=0.5) 
ax[1].plot((corona.rs-1)*constants.radius/1e6,np.mean(corona.temp,axis=(0,1)),'g.-',label = 'Mean Temp',markersize=1, lw=0.5) 
ax[2].plot((corona.rs-1)*constants.radius/1e6,np.mean(corona.bmag,axis=(0,1)),'k.-',label = 'Mean B',markersize = 1, lw=0.5) 
for axi in ax: 
    axi.legend(loc = 'upper right')
ax[-1].set_xlabel('Height [Mm]')
ax[0].set_xlim(1.,100)
ax[0].set_yscale('log')
ax[0].set_ylim(1e7,1e12)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(10000000.0, 1000000000000.0)

## **Initialize the pyCELP model of the line or lines to be synthesized**

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [7]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)
fe13_ln1 = fe13.get_emissionLine(10747.)
fe13_ln2 = fe13.get_emissionLine(10798.)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [8]:
fe13

pyCELP Ion class
    ---------------------
    Ion Name: fe_13
    Number of energy levels included: 50
    Number of SEE equations: 142
    Number of Radiative Transitions: 366
    Ionization Equilbrium Filename: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq

In [10]:
edens = 1e9
etemp = 1e7
ht = 0.1
thetab = 10.
%timeit fe13.calc_rho_sym(edens,etemp,ht, thetab)

520 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## **Calculate the statistical equilibrium for every cell**

- Here the statistical equilibrium is calculated for the Ion for each cell in the PSI model and the results of the upper level populations, alignments, and total ion population are stored in arrays. 
- The advantage of doing it this way is that that statistical equilibrium does not change with viewing angle, and so if you want to synthesis the polarized intensities at a different viewing angle, you do not have to recalculate the statistical equilibrium. 

In [11]:
nlons,nlats,nr = corona.bx.shape

###  Fe 13 

In [12]:
def work(argin): 
    
    ii,jj,kk = argin 
    
    edens = corona.ne[ii,jj,kk]
    etemp = corona.temp[ii,jj,kk]
    ht = (corona.rs[kk]-1).clip(1.e-8)
    thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
    
    fe13.calc_rho_sym(edens,etemp,ht, thetab)
    
    ln1 = fe13.get_emissionLine(10747.)
    ln2 = fe13.get_emissionLine(10798.)
    
    return (ii,jj,kk,ln1.C_coeff,ln1.upper_level_alignment, 
                ln2.C_coeff,ln2.upper_level_alignment)

## setup save file for repeat runs of this notebook and to not repeat the 
## statistical equilbrium calculation 

savefn = '/data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_fe13_1074_1079_pops.npz'

if not os.path.isfile(savefn):

    ncpus = multiprocessing.cpu_count()
    arg_array = []
    for ii in range(0,nlons,1): 
        for jj in range(0,nlats,1): 
            for kk in range(0,nr,1): 
                arg_array.append([ii,jj,kk])

    ccoef_10747 = np.zeros_like(corona.bx)
    align_10747 = np.zeros_like(corona.bx)
    ccoef_10798 = np.zeros_like(corona.bx)
    align_10798 = np.zeros_like(corona.bx)

    p     = multiprocessing.Pool(processes=ncpus) 
    rs    = p.imap(work,arg_array)
    p.close()

    for res in tqdm.tqdm(rs,total = len(arg_array)): 
        ii,jj,kk,r1,a1,r2,a2 = res 
        ccoef_10747[ii,jj,kk] = r1
        align_10747[ii,jj,kk] = a1
        ccoef_10798[ii,jj,kk] = r2
        align_10798[ii,jj,kk] = a2

    np.savez(savefn,ccoef_10747=ccoef_10747,align_10747=align_10747,ccoef_10798=ccoef_10798,align_10798=align_10798)
    
else: 
    
    ## if variables not defined load 
    if "ccoef_10747" not in globals():
        print('loading: ',savefn)
        npzd = np.load(savefn)
        ccoef_10747 = npzd['ccoef_10747']
        align_10747 = npzd['align_10747']
        ccoef_10798 = npzd['ccoef_10798']
        align_10798 = npzd['align_10798']

loading:  /data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_fe13_1074_1079_pops.npz


### do the same for Si X 1430 

In [13]:
si10 = pycelp.Ion('si_10',nlevels = 50)
si10_ln1 = si10.get_emissionLine(14300.)

 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_10/si_10.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_10/si_10.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_10/si_10.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_10/si_10.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [14]:
si10

pyCELP Ion class
    ---------------------
    Ion Name: si_10
    Number of energy levels included: 50
    Number of SEE equations: 107
    Number of Radiative Transitions: 435
    Ionization Equilbrium Filename: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq

In [15]:
def work(argin): 
    
    ii,jj,kk = argin 
    
    edens = corona.ne[ii,jj,kk]
    etemp = corona.temp[ii,jj,kk]
    ht = (corona.rs[kk]-1).clip(1.e-8)
    thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
    
    si10.calc_rho_sym(edens,etemp,ht, thetab)
    
    ln1 = si10.get_emissionLine(14301.)
    
    return (ii,jj,kk,ln1.C_coeff,ln1.upper_level_alignment)

## setup save file for repeat runs of this notebook and to not repeat the 
## statistical equilbrium calculation 

savefn = '/data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_si10_1430_pops.npz'

if not os.path.isfile(savefn):

    ncpus = multiprocessing.cpu_count()
    arg_array = []
    for ii in range(0,nlons,1): 
        for jj in range(0,nlats,1): 
            for kk in range(0,nr,1): 
                arg_array.append([ii,jj,kk])

    ccoef_14300 = np.zeros_like(corona.bx)
    align_14300 = np.zeros_like(corona.bx)

    p     = multiprocessing.Pool(processes=ncpus) 
    rs    = p.imap(work,arg_array)
    p.close()

    for res in tqdm.tqdm(rs,total = len(arg_array)): 
        ii,jj,kk,r1,a1 = res 
        ccoef_14300[ii,jj,kk] = r1
        align_14300[ii,jj,kk] = a1

    np.savez(savefn,ccoef_14300=ccoef_14300,align_14300=align_14300)
    
else: 
    
    ## if variables not defined load 
    if "ccoef_14300" not in globals():
        print('loading: ',savefn)
        npzd = np.load(savefn)
        ccoef_14300 = npzd['ccoef_14300']
        align_14300 = npzd['align_14300']

loading:  /data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_si10_1430_pops.npz


### do the same for Si IX 3934 

In [16]:
si9 = pycelp.Ion('si_9',nlevels = 50)
si9_ln1 = si9.get_emissionLine(39343.)

 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_9/si_9.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_9/si_9.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_9/si_9.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/si/si_9/si_9.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [17]:
si9

pyCELP Ion class
    ---------------------
    Ion Name: si_9
    Number of energy levels included: 50
    Number of SEE equations: 121
    Number of Radiative Transitions: 351
    Ionization Equilbrium Filename: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq

In [18]:
def work(argin): 
    
    ii,jj,kk = argin 
    
    edens = corona.ne[ii,jj,kk]
    etemp = corona.temp[ii,jj,kk]
    ht = (corona.rs[kk]-1).clip(1.e-8)
    thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
    
    si9.calc_rho_sym(edens,etemp,ht, thetab)
    
    ln1 = si9.get_emissionLine(39343.)
    
    return (ii,jj,kk,ln1.C_coeff,ln1.upper_level_alignment)

## setup save file for repeat runs of this notebook and to not repeat the 
## statistical equilbrium calculation 

savefn = '/data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_si10_3934_pops.npz'

if not os.path.isfile(savefn):

    ncpus = multiprocessing.cpu_count()
    arg_array = []
    for ii in range(0,nlons,1): 
        for jj in range(0,nlats,1): 
            for kk in range(0,nr,1): 
                arg_array.append([ii,jj,kk])

    ccoef_39343 = np.zeros_like(corona.bx)
    align_39343 = np.zeros_like(corona.bx)

    p     = multiprocessing.Pool(processes=ncpus) 
    rs    = p.imap(work,arg_array)
    p.close()

    for res in tqdm.tqdm(rs,total = len(arg_array)): 
        ii,jj,kk,r1,a1 = res 
        ccoef_39343[ii,jj,kk] = r1
        align_39343[ii,jj,kk] = a1

    np.savez(savefn,ccoef_39343=ccoef_39343,align_39343=align_39343)
    
else: 
    
    ## if variables not defined load 
    if "ccoef_39343" not in globals():
        print('loading: ',savefn)
        npzd = np.load(savefn)
        ccoef_39343 = npzd['ccoef_39343']
        align_39343 = npzd['align_39343']

loading:  /data/tschad/psiModels/cr2254-high/hmi_mast_mas_std_0101/cr2254_high_0101_si10_3934_pops.npz


## **Setup forward synthesis geometry**

- This is set up to synthesize a FOV from an arbitrary observer's position, def

### Observer's Position

In [19]:
## not yet sure what to use here?  How to get fractional carrington rotation of the PSI data?
crn        = sun.carrington_rotation_number('2022-02-14 20:00:00.000')
crt        = sun.carrington_rotation_time(crn)
obsLon     = sun.L0(crt).rad
obsLat     = sun.B0(crt).rad
Obs_Sun_AU = sun.earth_distance(crt).value
rsunarc    = sun.angular_radius(crt).value  ## radius of sun in arcseconds .. later replace with sun ephemeris

print('Carrington Rotation Number:     ',crn)
print('Carrington Rotation Time:       ',crt)
print('Observers Longitude [deg]:      ',np.rad2deg(obsLon))
print('Observers Latitude [deg]:       ',np.rad2deg(obsLat))
print('Observers distance to Sun [AU]: ',Obs_Sun_AU)
print('Apparent solar radius [arcsec]: ',rsunarc)

Carrington Rotation Number:      2254.2438864703195
Carrington Rotation Time:        2022-02-14 19:59:59.945
Observers Longitude [deg]:       272.2008791250737
Observers Latitude [deg]:        -6.815283900001111
Observers distance to Sun [AU]:  0.9875632519781533
Apparent solar radius [arcsec]:  971.3112266968824


In [20]:
solar_rad_m, au_m = constants.radius.value,constants.astronomical_unit.value
rObs     = Obs_Sun_AU * (au_m/solar_rad_m)  ## rObs is the radial position of the observer in solar radii
thetaObs = np.pi/2. - obsLat   ## Observers latitude in the PSI model geometry (0 is the NorthPole)
phiObs   = obsLon ## Observers Lon in PSI model geometry  
## cartesian position of Observer in PSI model geometery (units of solar radii)
xObs,yObs,zObs = rObs*np.sin(thetaObs)*np.cos(phiObs),rObs*np.sin(thetaObs)*np.sin(phiObs),rObs*np.cos(thetaObs)
print(xObs,yObs,zObs)

8.097583112278484 -210.70176686233026 -25.200260731789186


### Synthesized Field-of-View Size and Sampling

- The synthesis geometry will have the x axis pointed toward the observer.  Z is aligned with polar axis. 

In [21]:
yfov_range = (-2.*rsunarc,0) # (-3.*rsunarc,+3*rsunarc)
zfov_range = (-0.25*rsunarc,2.*rsunarc) # (-3.*rsunarc,+3*rsunarc)
arcsamp    = 10.  ## sampling in arcsecond

#yfov_range = (-3.*rsunarc,+3*rsunarc)
#zfov_range = (-3.*rsunarc,+3*rsunarc)
#arcsamp    = 50.  ## sampling in arcsecond

nysamp = int(np.ceil((yfov_range[1]-yfov_range[0])/arcsamp))
nzsamp = int(np.ceil((zfov_range[1]-zfov_range[0])/arcsamp))
yarc = np.linspace(yfov_range[0],yfov_range[1],nysamp)
zarc = np.linspace(zfov_range[0],zfov_range[1],nzsamp)
zza,yya  = np.meshgrid(zarc,yarc,indexing = 'ij')
rra = np.sqrt(yya**2. + zza**2.) 
mask_ondisk = 1.*(rra>rsunarc)    ## a mask where the field of view cross the solar disk

In [22]:
## First find the cartestian points of the reference point for the line of sight
## in the observer's geometry.  For offlimb points, this is the plane of the sky (yz). 
## On disk, these points should lie on the solar surface
xxObs = np.zeros_like(yya)
yyObs = rObs * np.tan(np.deg2rad(yya/3600.))
zzObs = rObs * np.tan(np.deg2rad(zza/3600.))
rrObs = np.sqrt(yyObs**2 + zzObs**2)
xxObs[rrObs<1] = np.sqrt(1 - rrObs[rrObs<1]**2) 
plt.figure(figsize = (2.5,2))
plt.imshow(xxObs)
plt.title('Starting X point')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Starting X point')

In [23]:
## rotate these points into the model geometry with Euler rotation
rotm = euler_ry((thetaObs-np.pi/2.)) @ euler_rz(phiObs)
## earlier I had the rotation matrix as the transpose...not sure why..incorrect?
xyz_model = rotm  @ np.stack((xxObs.flatten(),yyObs.flatten(),zzObs.flatten())) 

## NOW WITH THE XYZ_MODEL POINTS AND THE LOCATION OF THE OBSERVER IN MODEL FRAME
## come up with the parametric equations for the los of sight
losvec    = np.stack((xObs-xyz_model[0,:],yObs-xyz_model[1,:],zObs - xyz_model[2,:]))
losveclen = np.linalg.norm(losvec,axis=0,ord=2,keepdims = True)
losvec    = losvec / losveclen
startpt   = xyz_model

In [24]:
lons,lats,rs = corona.lons,corona.lats,corona.rs
deltaR = (np.roll(rs,-1) - rs)[:(-1)]   
deltaR = np.cumsum(deltaR)
deltaR = np.hstack((-np.flip(deltaR),deltaR))  ## both directions

In [25]:
rays_xyz = startpt[None,:,:] + (losvec * deltaR[:,None,None])
rays_xyz = np.swapaxes(rays_xyz,1,2)

In [26]:
nrays   = startpt.shape[1]
ray_npt = len(deltaR)
print('Number of rays to trace: ',nrays) 
print('Max number of samples per ray: ',ray_npt)

Number of rays to trace:  42705
Max number of samples per ray:  506


In [27]:
## get spherical coordinates of rays
rays_sph = np.zeros_like(rays_xyz)
rays_sph[:,:,0] = np.linalg.norm(rays_xyz,axis=2)        ## l2-norm 
rays_sph[:,:,1] = np.arccos(rays_xyz[:,:,2] / rays_sph[:,:,0])   ## arccos (z / r) 
rays_sph[:,:,2] = np.arctan2(rays_xyz[:,:,1] , rays_xyz[:,:,0])   ## arctan (y / x) 
rays_sph[:,:,2][rays_sph[:,:,2] <0] += 2.*np.pi ## so that it falls in the simulation domain 

In [28]:
print(startpt.shape,rays_sph.shape,rays_xyz.shape)
print(np.max(deltaR))

(3, 42705) (506, 42705, 3) (506, 42705, 3)
29.0


In [29]:
if 0: 
    print(rays_sph.shape)
    plt.figure()
    ax = plt.axes(projection='3d')
    ax.scatter3D(rays_xyz[0,::10,0],rays_xyz[0,::10,1],rays_xyz[0,::10,2])
    ax.scatter3D(rays_xyz[100,::10,0],rays_xyz[100,::10,1],rays_xyz[100,::10,2])
    ax.scatter3D(rays_xyz[-1,::10,0],rays_xyz[-1,::10,1],rays_xyz[-1,::10,2])

    ax.scatter3D(xObs,yObs,zObs,'x')
    ax.plot3D([0,xObs],[0,yObs],[0,zObs])
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.set_zlim(-5,5)

    plt.figure(figsize = (6,4))
    plt.scatter(rays_xyz[ray_npt//2,:,2], rays_xyz[ray_npt//2,:,1],c=rays_sph[ray_npt//2,:,0],s=0.5)  ## lons, lats
    plt.ylim(-1,1)

## **Interpolate model variables at ray samples**

In [ ]:
rays_xyz = rays_xyz.reshape(ray_npt*nrays,3)
rays_sph = rays_sph.reshape(ray_npt*nrays,3)

In [31]:
#### Get interpolating functions for all necessary simulation data
#### the ccoefInt already has the density factors included
ccoef1Int = rgi((lons,lats,rs),ccoef_10747,method = 'linear',fill_value = 0.,bounds_error = False)
align1Int = rgi((lons,lats,rs),align_10747,method = 'linear',fill_value = 0.,bounds_error = False)
ccoef2Int = rgi((lons,lats,rs),ccoef_10798,method = 'linear',fill_value = 0.,bounds_error = False)
align2Int = rgi((lons,lats,rs),align_10798,method = 'linear',fill_value = 0.,bounds_error = False)
ccoef3Int = rgi((lons,lats,rs),ccoef_14300,method = 'linear',fill_value = 0.,bounds_error = False)
align3Int = rgi((lons,lats,rs),align_14300,method = 'linear',fill_value = 0.,bounds_error = False)
ccoef4Int = rgi((lons,lats,rs),ccoef_39343,method = 'linear',fill_value = 0.,bounds_error = False)
align4Int = rgi((lons,lats,rs),align_39343,method = 'linear',fill_value = 0.,bounds_error = False)
bxInt    = rgi((lons,lats,rs),corona.bx,method = 'linear',fill_value = 0.,bounds_error = False)
byInt    = rgi((lons,lats,rs),corona.by,method = 'linear',fill_value = 0.,bounds_error = False)
bzInt    = rgi((lons,lats,rs),corona.bz,method = 'linear',fill_value = 0.,bounds_error = False)
tempInt  = rgi((lons,lats,rs),corona.temp,method = 'linear',fill_value = 0.,bounds_error = False)
neInt    = rgi((lons,lats,rs),corona.ne,method = 'linear',fill_value = 0.,bounds_error = False)

In [32]:
## get bx,by,bz in PSI geometry for all the ray points
## this is the step that takes the longest! 
%time bx_rays = bxInt(rays_sph[:,::-1])
%time by_rays = byInt(rays_sph[:,::-1]) 
%time bz_rays = bzInt(rays_sph[:,::-1]) 
%time temp_rays = tempInt(rays_sph[:,::-1])
%time ne_rays = neInt(rays_sph[:,::-1])
print('1')
%time ccoef1_rays = ccoef1Int(rays_sph[:,::-1]) 
%time align1_rays = align1Int(rays_sph[:,::-1]) 
print('2')
%time ccoef2_rays = ccoef2Int(rays_sph[:,::-1]) 
%time align2_rays = align2Int(rays_sph[:,::-1]) 
print('3')
%time ccoef3_rays = ccoef3Int(rays_sph[:,::-1]) 
%time align3_rays = align3Int(rays_sph[:,::-1]) 
print('4')
%time ccoef4_rays = ccoef4Int(rays_sph[:,::-1]) 
%time align4_rays = align4Int(rays_sph[:,::-1]) 

CPU times: user 6.48 s, sys: 2.41 s, total: 8.89 s
Wall time: 8.91 s
CPU times: user 6.48 s, sys: 2.38 s, total: 8.86 s
Wall time: 8.89 s
CPU times: user 6.47 s, sys: 2.41 s, total: 8.88 s
Wall time: 8.91 s
CPU times: user 6.54 s, sys: 2.33 s, total: 8.87 s
Wall time: 8.9 s
CPU times: user 6.5 s, sys: 2.38 s, total: 8.89 s
Wall time: 8.91 s
1
CPU times: user 6.5 s, sys: 2.38 s, total: 8.88 s
Wall time: 8.9 s
CPU times: user 6.5 s, sys: 2.36 s, total: 8.86 s
Wall time: 8.88 s
2
CPU times: user 6.48 s, sys: 2.39 s, total: 8.88 s
Wall time: 8.9 s
CPU times: user 6.52 s, sys: 2.34 s, total: 8.87 s
Wall time: 8.89 s
3
CPU times: user 6.48 s, sys: 2.4 s, total: 8.88 s
Wall time: 8.9 s
CPU times: user 6.5 s, sys: 2.38 s, total: 8.88 s
Wall time: 8.9 s
4
CPU times: user 6.46 s, sys: 2.42 s, total: 8.88 s
Wall time: 8.91 s
CPU times: user 6.49 s, sys: 2.4 s, total: 8.89 s
Wall time: 8.91 s


In [33]:
ccoef1_rays = ccoef1_rays.reshape(ray_npt,nrays)
align1_rays = align1_rays.reshape(ray_npt,nrays)
ccoef2_rays = ccoef2_rays.reshape(ray_npt,nrays)
align2_rays = align2_rays.reshape(ray_npt,nrays)
ccoef3_rays = ccoef3_rays.reshape(ray_npt,nrays)
align3_rays = align3_rays.reshape(ray_npt,nrays)
ccoef4_rays = ccoef4_rays.reshape(ray_npt,nrays)
align4_rays = align4_rays.reshape(ray_npt,nrays)
temp_rays = temp_rays.reshape(ray_npt,nrays)
ne_rays = ne_rays.reshape(ray_npt,nrays)

In [97]:
rays_xyz = rays_xyz.reshape(ray_npt,nrays,3)
rays_sph = rays_sph.reshape(ray_npt,nrays,3)

In [98]:
## calculate the magnetic field inclination angle wrt the line-of-sight 
bxyzs = np.stack((bx_rays,by_rays,bz_rays)).T
blens = np.linalg.norm(bxyzs,axis =1)
bxyzs = bxyzs.reshape(ray_npt,nrays,3)
blens = blens.reshape(ray_npt,nrays)
thetaBlos = np.arccos( (np.sum(bxyzs*losvec.T[None,:,:],axis=2)/blens).clip(min = -1,max=1))
thetaBlos[np.isnan(thetaBlos)] = 0.

/tmp/ipykernel_1091557/3178097367.py:6: RuntimeWarning: invalid value encountered in true_divide
  thetaBlos = np.arccos( (np.sum(bxyzs*losvec.T[None,:,:],axis=2)/blens).clip(min = -1,max=1))


In [99]:
## angle between losvec and the vector from disk center
thetaDClos = np.arccos( (np.sum(rays_xyz*losvec.T[None,:,:],axis=2)/rays_sph[:,:,0]).clip(min = -1,max=1))

In [100]:
## get projection of B onto plane perpendicular to LOS
## and the projection of DC vector onto same plane
Bperp  = bxyzs - (blens*np.cos(thetaBlos))[:,:,None] * losvec.T[None,:,:]
DCperp = rays_xyz - (rays_sph[:,:,0]*np.cos(thetaDClos))[:,:,None] * losvec.T[None,:,:]

In [101]:
## find angle between Bperp and DCperp, which is the azimuthal angle relative to disk center
costhetaAzi = np.sum((Bperp*DCperp),axis=2)/(np.linalg.norm(Bperp,axis = 2)*np.linalg.norm(DCperp,axis = 2))
del Bperp
del DCperp
thetaAzi = np.arccos(costhetaAzi.clip(min =-1,max=1))
thetaAzi[np.isnan(thetaAzi)] = 0.
del costhetaAzi

/tmp/ipykernel_1091557/3888707345.py:2: RuntimeWarning: invalid value encountered in true_divide
  costhetaAzi = np.sum((Bperp*DCperp),axis=2)/(np.linalg.norm(Bperp,axis = 2)*np.linalg.norm(DCperp,axis = 2))


In [102]:
fig,ax = plt.subplots(2,3,figsize = (10,7),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(bx_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))
im1 = ax[1].imshow(by_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))
im2 = ax[2].imshow(bz_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape))

im3 = ax[3].imshow(blens[ray_npt//2,:].reshape(*yya.shape))
im4 = ax[4].imshow(thetaBlos[ray_npt//2,:].reshape(*yya.shape))
im5 = ax[5].imshow(thetaAzi[ray_npt//2,:].reshape(*yya.shape))
for im in im0,im1,im2,: im.set_clim(-4,4)

labs = 'Bx (POS)','By (POS)','Bz (POS)','B (POS)','ThetaBLOS (POS)','Bazimuth (POS)'
for n in range(6): ax[n].set_title(labs[n])

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nMagnetic Field in the Plane of the Sky from the Observer\nComponents are in the PSI model frame') 

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
fig,ax = plt.subplots(1,2,figsize = (7,4),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(temp_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape)*mask_ondisk,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
im1 = ax[1].imshow(ne_rays.reshape(ray_npt,nrays)[ray_npt//2,:].reshape(*yya.shape)*mask_ondisk,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))

print(10.**6.25)

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle('POS Temperature and Electron Density')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1778279.410038923


## Forward Synthesize the Thomson Scattering

In [126]:
ht = (rays_sph[:,:,0]-1).clip(1.e-8) * constants.radius.value/1000.
ht = ht.reshape(ray_npt*nrays)
J00 = allen.J00_sym(10747.,ht)
units = J00[1]
print(units)
J00 = J00[0]
omega = allen.get_omega(10747.,ht)
Idc, Idc_unit = allen.i_nu(10747.,1.)

erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ Hz$^{-1}$


In [127]:
J00 = J00.reshape(ray_npt,nrays)
omega = omega.reshape(ray_npt,nrays)

In [132]:
cc = 0.665e-24  ## total thomson scattering crosssection
eps_rays = cc * ne_rays * J00 * (1 + omega/2 - (3./4.)*omega*np.sin(thetaDClos)**2)
Ithomson = np.trapz(eps_rays,dray,axis=0).reshape(*yya.shape) * mask_ondisk / Idc *1e6
eps_rays = - 3./4. * cc * ne_rays * omega * J00 * np.sin(thetaDClos)**2
pBthomson = np.trapz(eps_rays,dray,axis=0).reshape(*yya.shape) * mask_ondisk / Idc *1e6

In [138]:
fig,ax = plt.subplots(1,3,figsize = (10,3.) )
ax = ax.flatten()
im0 = ax[0].imshow(Ithomson,norm = LogNorm(),cmap = plt.get_cmap('viridis'))
im1 = ax[1].imshow(-pBthomson,norm = LogNorm(),cmap = plt.get_cmap('viridis'))
im2 = ax[2].imshow(-pBthomson/Ithomson,cmap = plt.get_cmap('viridis'))
im0.set_clim(0.001,1)
im1.set_clim(0.001,1)
im2.set_clim(0.2,0.7)

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

ax[0].set_title('Thomson I log mills')
ax[1].set_title('Thomson pB log mills')
ax[2].set_title('pB/I')

fig.suptitle('Thomson Scattered Radiation at 1074 nm')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_1091557/2776832075.py:5: RuntimeWarning: invalid value encountered in true_divide
  im2 = ax[2].imshow(-pBthomson/Ithomson,cmap = plt.get_cmap('viridis'))


## **Forward Synthesize Emission Line** 

In [43]:
## distance vector along the ray 
dray = deltaR[:,None] * constants.radius.value*100.  ## cm 
drayg = np.gradient(dray[:,0])

In [49]:
geff,D_coeff,E_coeff = fe13_ln1.geff,fe13_ln1.Dcoeff,fe13_ln1.Ecoeff
epsI   = ccoef1_rays*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*align1_rays)
epsQnr = ccoef1_rays*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*align1_rays
epsQ   = np.cos(2.*thetaAzi)*epsQnr
epsU   = -np.sin(2.*thetaAzi)*epsQnr
epsV   = ccoef1_rays*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*align1_rays)
epsI_int1 = np.trapz(epsI,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsQ_int1 = np.trapz(epsQ,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsU_int1 = np.trapz(epsU,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsV_int1 = np.trapz(epsV,dray,axis=0).reshape(*yya.shape) * mask_ondisk
ne_wgtI1 = np.sum(epsI*ne_rays*drayg[:,None],axis=0)    ## volumetric weighted? 
ne_wgtI1 /= np.sum(epsI*drayg[:,None],axis=0)
ne_wgtI1 = ne_wgtI1.reshape(*yya.shape)*mask_ondisk

In [50]:
geff,D_coeff,E_coeff = fe13_ln2.geff,fe13_ln2.Dcoeff,fe13_ln2.Ecoeff
epsI   = ccoef2_rays*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*align2_rays)
epsQnr = ccoef2_rays*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*align2_rays
epsQ   = np.cos(2.*thetaAzi)*epsQnr
epsU   = -np.sin(2.*thetaAzi)*epsQnr
epsV   = ccoef2_rays*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*align2_rays)
epsI_int2 = np.trapz(epsI,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsQ_int2 = np.trapz(epsQ,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsU_int2 = np.trapz(epsU,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsV_int2 = np.trapz(epsV,dray,axis=0).reshape(*yya.shape) * mask_ondisk
ne_wgtI2 = np.sum(epsI*ne_rays*drayg[:,None],axis=0)    ## volumetric weighted? 
ne_wgtI2 /= np.sum(epsI*drayg[:,None],axis=0)
ne_wgtI2 = ne_wgtI2.reshape(*yya.shape)*mask_ondisk

In [51]:
geff,D_coeff,E_coeff = si10_ln1.geff,si10_ln1.Dcoeff,si10_ln1.Ecoeff
epsI   = ccoef3_rays*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*align3_rays)
epsQnr = ccoef3_rays*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*align3_rays
epsQ   = np.cos(2.*thetaAzi)*epsQnr
epsU   = -np.sin(2.*thetaAzi)*epsQnr
epsV   = ccoef3_rays*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*align3_rays)
epsI_int3 = np.trapz(epsI,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsQ_int3 = np.trapz(epsQ,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsU_int3 = np.trapz(epsU,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsV_int3 = np.trapz(epsV,dray,axis=0).reshape(*yya.shape) * mask_ondisk
ne_wgtI3 = np.sum(epsI*ne_rays*drayg[:,None],axis=0)    ## volumetric weighted? 
ne_wgtI3 /= np.sum(epsI*drayg[:,None],axis=0)
ne_wgtI3 = ne_wgtI3.reshape(*yya.shape)*mask_ondisk

In [52]:
geff,D_coeff,E_coeff = si9_ln1.geff,si9_ln1.Dcoeff,si9_ln1.Ecoeff
epsI   = ccoef4_rays*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*align4_rays)
epsQnr = ccoef4_rays*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*align4_rays
epsQ   = np.cos(2.*thetaAzi)*epsQnr
epsU   = -np.sin(2.*thetaAzi)*epsQnr
epsV   = ccoef4_rays*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*align4_rays)
epsI_int4 = np.trapz(epsI,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsQ_int4 = np.trapz(epsQ,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsU_int4 = np.trapz(epsU,dray,axis=0).reshape(*yya.shape) * mask_ondisk
epsV_int4 = np.trapz(epsV,dray,axis=0).reshape(*yya.shape) * mask_ondisk
ne_wgtI4 = np.sum(epsI*ne_rays*drayg[:,None],axis=0)    ## volumetric weighted? 
ne_wgtI4 /= np.sum(epsI*drayg[:,None],axis=0)
ne_wgtI4 = ne_wgtI4.reshape(*yya.shape)*mask_ondisk

In [56]:
fig,ax = plt.subplots(1,4,figsize = (12,3.) )
ax = ax.flatten()
ax[0].imshow(ne_wgtI1,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
ax[1].imshow(ne_wgtI2,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
ax[2].imshow(ne_wgtI3,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
ax[3].imshow(ne_wgtI4,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))

for axi in ax:
    imn = axi.get_images()[0]
    imn.set_clim(1e5,1e9)
    
cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [194]:
fig,ax = plt.subplots(1,4,figsize = (12,3.) )
ax = ax.flatten()

sr2arcsec = (180./np.pi)**2.*3600.**2.
hh        = 6.6260755e-27                  ## erg s

phergs1    = hh*(3.e8)/(10747.0e-10)      ## ergs to photons
ax[0].imshow(epsI_int1/sr2arcsec/phergs1,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
phergs2    = hh*(3.e8)/(10798.0e-10)      ## ergs to photons
ax[1].imshow(epsI_int2/sr2arcsec/phergs2,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
phergs3    = hh*(3.e8)/(14300.0e-10)      ## ergs to photons
ax[2].imshow(epsI_int3/sr2arcsec/phergs3,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))
phergs4    = hh*(3.e8)/(39340.0e-10)      ## ergs to photons
ax[3].imshow(epsI_int4/sr2arcsec/phergs4,norm = LogNorm(),cmap = plt.get_cmap('nipy_spectral'))

for axi in ax:
    imn = axi.get_images()[0]
    imn.set_clim(0.001,1000)
    
cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
fig,ax = plt.subplots(1,3,figsize = (9,3))
ax =ax.flatten()
im0 = ax[0].imshow(epsI_int2/epsI_int1)
im1 = ax[1].imshow(ne_wgtI1,norm = LogNorm())
im1.set_clim(1e4,1e10)

ax[2].plot(ne_wgtI1.flatten(),(epsI_int2/epsI_int1).flatten(),'.')
ax[2].set_xlim(1e4,1e10)
ax[2].set_ylim(0,1.0)
ax[2].set_xscale('log')
## add density curve here

cbars = []
for axi in ax[0:2]:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_1091557/1725698819.py:3: RuntimeWarning: invalid value encountered in true_divide
  im0 = ax[0].imshow(epsI_int2/epsI_int1)
/tmp/ipykernel_1091557/1725698819.py:7: RuntimeWarning: invalid value encountered in true_divide
  ax[2].plot(ne_wgtI1.flatten(),(epsI_int2/epsI_int1).flatten(),'.')


In [142]:
fig,ax = plt.subplots(2,3,figsize = (10,7),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(np.log10(epsI_int1))
im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int1**2. + epsU_int1**2)))
im2 = ax[2].imshow(np.sqrt(epsQ_int1**2. + epsU_int1**2)/epsI_int1)
im3 = ax[3].imshow(epsV_int1)
mrang = np.max(np.abs(np.percentile(epsV_int1,[1,99])))
im3.set_clim(-mrang,mrang)

im4 = ax[4].imshow(0.5*np.arctan2(epsU_int1 , epsQ_int1)*mask_ondisk)

labs = '1074 Log I','Log LinPol','LinPolFrac','epsV integ.','Azimuth Angle'
for n in range(5): ax[n].set_title(labs[n])

ax[-1].axis('off')

cbars = []
for axi in ax[:-1]:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nLOS-integrated polarized emission coefficients') 

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_1091557/1762355804.py:3: RuntimeWarning: divide by zero encountered in log10
  im0 = ax[0].imshow(np.log10(epsI_int1))
/tmp/ipykernel_1091557/1762355804.py:4: RuntimeWarning: divide by zero encountered in log10
  im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int1**2. + epsU_int1**2)))
/tmp/ipykernel_1091557/1762355804.py:5: RuntimeWarning: invalid value encountered in true_divide
  im2 = ax[2].imshow(np.sqrt(epsQ_int1**2. + epsU_int1**2)/epsI_int1)


In [143]:
fig,ax = plt.subplots(2,3,figsize = (10,7),sharex=True,sharey=True)
ax = ax.flatten()
im0 = ax[0].imshow(np.log10(epsI_int4))
im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int4**2. + epsU_int4**2)))
im2 = ax[2].imshow(np.sqrt(epsQ_int4**2. + epsU_int4**2)/epsI_int4)
im3 = ax[3].imshow(epsV_int4)
mrang = np.max(np.abs(np.percentile(epsV_int4,[1,99])))
im3.set_clim(-mrang,mrang)

im4 = ax[4].imshow(0.5*np.arctan2(epsU_int4 , epsQ_int4)*mask_ondisk)

labs = '3934 Log I','Log LinPol','LinPolFrac','epsV integ.','Azimuth Angle'
for n in range(5): ax[n].set_title(labs[n])

ax[-1].axis('off')

cbars = []
for axi in ax[:-1]:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nLOS-integrated polarized emission coefficients') 

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_1091557/2515412780.py:3: RuntimeWarning: divide by zero encountered in log10
  im0 = ax[0].imshow(np.log10(epsI_int4))
/tmp/ipykernel_1091557/2515412780.py:4: RuntimeWarning: divide by zero encountered in log10
  im1 = ax[1].imshow(np.log10(np.sqrt(epsQ_int4**2. + epsU_int4**2)))
/tmp/ipykernel_1091557/2515412780.py:5: RuntimeWarning: invalid value encountered in true_divide
  im2 = ax[2].imshow(np.sqrt(epsQ_int4**2. + epsU_int4**2)/epsI_int4)


In [144]:
sr2arcsec = (180./np.pi)**2.*3600.**2.
hh        = 6.6260755e-27                  ## erg s
phergs    = hh*(3.e8)/(10747.0e-10)      ## ergs to photons

epsI_int = np.copy(epsI_int1)

plt.figure()
plt.imshow((epsI_int/sr2arcsec/phergs), norm=LogNorm(), extent = (yya[0,0],yya[0,-1],zza[0,0],zza[-1,0]),cmap = plt.get_cmap('nipy_spectral'))
#plt.imshow((epsI_int/sr2arcsec/phergs),  extent = (yya[0,0],yya[0,-1],zza[0,0],zza[-1,0]))
plt.clim(0.01,1000)

rsun =  971.975455850595
ypt1 =  (1.15*rsun-117.5) * np.cos(np.deg2rad(65.))
xpt1 = -(1.15*rsun-117.5) * np.sin(np.deg2rad(65.))
ypt2 =  (1.15*rsun+117.5) * np.cos(np.deg2rad(65.))
xpt2 = -(1.15*rsun+117.5) * np.sin(np.deg2rad(65.))
plt.plot([xpt1,xpt2],[ypt1,ypt2],color = 'black',lw = 2)

ypt1 =  (1.4*rsun-117.5) * np.cos(np.deg2rad(65.))
xpt1 = -(1.4*rsun-117.5) * np.sin(np.deg2rad(65.))
ypt2 =  (1.4*rsun+117.5) * np.cos(np.deg2rad(65.))
xpt2 = -(1.4*rsun+117.5) * np.sin(np.deg2rad(65.))
plt.plot([xpt1,xpt2],[ypt1,ypt2],color = 'white',lw = 2)

plt.colorbar()
plt.title(' Fe XIII 1074.7 total line integrated brightness') 

plt.autoscale(False)
th = np.linspace(0,2.*np.pi,100)
for rr in 1.1,1.2,1.3,1.5,:
    plt.plot(rr*rsun*np.cos(th),rr*rsun*np.sin(th),color = 'white',lw = 0.8,linestyle = 'dotted',clip_on = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Extract values along radial slit

In [158]:
## do this for all the models and plot the radial curves of all line strengths, AIAs, and pB.  
## then get KCOR measurements, AIA measurements, and DKIST measurements. 

In [231]:
img = (epsI_int4/sr2arcsec/phergs4)
print(img.shape,zza[:,0].shape)
iint = rgi((zza[:,0],yya[0,:]),img,method = 'linear',fill_value = 0.,bounds_error = False)

(219, 195) (219,)


In [232]:
yrad = -np.sin(np.deg2rad(65.)) * np.linspace(1.02,2.,200) * rsun
zrad =  np.cos(np.deg2rad(65.)) * np.linspace(1.02,2.,200) * rsun
zy = np.zeros((len(yrad),2))
zy[:,0] = zrad
zy[:,1] = yrad

yrad2 = -np.sin(np.deg2rad(50.)) * np.linspace(1.02,2.,200) * rsun
zrad2 =  np.cos(np.deg2rad(50.)) * np.linspace(1.02,2.,200) * rsun
zy2 = np.zeros((len(yrad),2))
zy2[:,0] = zrad2
zy2[:,1] = yrad2

plt.figure()
plt.imshow(img, norm=LogNorm(), extent = (yya[0,0],yya[0,-1],zza[0,0],zza[-1,0]),cmap = plt.get_cmap('nipy_spectral'))
plt.clim(0.01,1000)
plt.plot(yrad,zrad,'.')
plt.plot(yrad2,zrad2,'--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [233]:
iint((850,-1770))

array(0.14833377)

In [234]:
iint1 = rgi((zza[:,0],yya[0,:]),(epsI_int1/sr2arcsec/phergs1),method = 'linear',fill_value = 0.,bounds_error = False)
iint2 = rgi((zza[:,0],yya[0,:]),(epsI_int2/sr2arcsec/phergs2),method = 'linear',fill_value = 0.,bounds_error = False)
iint3 = rgi((zza[:,0],yya[0,:]),(epsI_int3/sr2arcsec/phergs3),method = 'linear',fill_value = 0.,bounds_error = False)
iint4 = rgi((zza[:,0],yya[0,:]),(epsI_int4/sr2arcsec/phergs4),method = 'linear',fill_value = 0.,bounds_error = False)

iintT = rgi((zza[:,0],yya[0,:]),Ithomson,method = 'linear',fill_value = 0.,bounds_error = False)
iintpB = rgi((zza[:,0],yya[0,:]),-pBthomson,method = 'linear',fill_value = 0.,bounds_error = False)

In [249]:
fig,ax = plt.subplots(1,2,figsize = (8,4.25),sharex=True)
fig.suptitle('Note scale factor')

ax = ax.flatten()
scl = 3.96
ax[0].plot(np.linspace(1.02,2.,200),scl*iint1(zy),'.-',label = 'Fe XIII 1074')
ax[0].plot(np.linspace(1.02,2.,200),scl*iint1(zy2),'--',label = 'Fe XIII 1074')
ax[0].plot(np.linspace(1.02,2.,200),scl*iint2(zy),'.-',label = 'Fe XIII 1079')
ax[0].plot(np.linspace(1.02,2.,200),scl*iint3(zy),'.-',label = 'Si X 1430')
ax[0].plot(np.linspace(1.02,2.,200),scl*iint4(zy),'.-',label = 'Si IX 3934')
ax[0].set_yscale('log')
ax[0].legend()
ax[0].set_ylabel('photons cm^-2 s^-1 arcsec^-2')
ax[0].set_xlabel('Distance from Disk Center')

ax[0].plot([1.,2],[1200,1200])
ax[0].plot([1.,2],[400,400])
ax[0].plot([1.,2],[25,25])

ax[1].plot(np.linspace(1.02,2.,200),iintT(zy),'.-',label = 'Thomson I mill 1074')
ax[1].plot(np.linspace(1.02,2.,200),iintpB(zy),'.-',label = 'Thomson pB mill 1074')
ax[1].set_yscale('log')
ax[1].legend()
ax[1].set_xlabel('Distance from Disk Center')


for z in range(3): fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##  AIA emission 

In [252]:
idls = readsav('./aia_resp_database/aia_resp_temp_dens_archive.sav')
print(idls.keys())
aia_tresp = idls['aia_tresp'].copy()
aia_temp = idls['aia_temp'].copy()
aia_dens = idls['aia_dens'].copy()
idls['ieqf'],idls['abf']
chnames = idls['chnames'].copy()
print(chnames)  ## is it opposite
print(np.log10(aia_dens))

dict_keys(['aia_tresp', 'aia_temp', 'aia_dens', 'ieqf', 'abf', 'chnames'])
[b'94' b'131' b'171' b'193' b'211' b'304' b'335']
[ 5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11. ]


In [253]:
## interpolate the response in log space

In [254]:
aia_tresp_int = rgi((np.arange(7),np.log10(aia_dens),np.log10(aia_temp)),aia_tresp,method = 'linear',fill_value = None,bounds_error = False)

In [255]:
cc = np.stack((corona.ne.flatten()*0+0,np.log10(corona.ne.flatten()),np.log10(corona.temp.flatten()))).T
cc.shape

(10784332, 3)

In [256]:
aia_tresp.shape,aia_temp.shape,aia_dens.shape

((7, 13, 81), (81,), (13,))

In [257]:
plt.figure()
for n in range(7): 
    p1, = plt.plot(aia_temp,aia_tresp[n,0,:]*3.96,label = chnames[n].decode('UTF-8'))
    for d in range(1,13): 
        plt.plot(aia_temp,aia_tresp[n,d,:]*3.96,color = p1.get_color())
        
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.xlim(10**5,10**7.8)
plt.ylim(0.5e-28,1e-23)
plt.title('Missing factor of 3.9 or so due to Iron abundance\n See Del Zanna et al. 2011') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Missing factor of 3.9 or so due to Iron abundance\n See Del Zanna et al. 2011')

In [258]:
imgs = np.zeros((7,*yya.shape))

In [259]:
for ch in tqdm.tqdm(range(7)): 
    print(' channel:  ',ch,chnames[ch].decode()) 
    cc[:,0] = ch
    r0 = aia_tresp_int(cc)
    aiaEps = r0.reshape(corona.ne.shape)
    epsInt = rgi((lons,lats,rs),aiaEps,method = 'linear',fill_value = 0,bounds_error = False)
    eps_rays = epsInt(rays_sph[:,::-1])   ## reverse because the interpolation is lons,lats, rs 
    
    ## zero out epsilon behind the Sun in observer's geometry 
    eps_rays = eps_rays.reshape(ray_npt,*yya.shape)
    eps_rays[:(ray_npt//2),:,:]  = eps_rays[:(ray_npt//2),:,:] * (rrObs>1)[None,:,:]
    eps_rays = eps_rays.reshape(ray_npt,nrays)
    
    epsI_int = np.trapz(eps_rays*ne_rays**2,dray,axis=0).reshape(*yya.shape) 
    
    imgs[ch,:,:] = epsI_int

  0%|                                                                                                                                                                                                                                                          | 0/7 [00:00<?, ?it/s]

 channel:   0 94


 14%|██████████████████████████████████▌                                                                                                                                                                                                               | 1/7 [00:17<01:45, 17.66s/it]

 channel:   1 131


 29%|█████████████████████████████████████████████████████████████████████▏                                                                                                                                                                            | 2/7 [00:35<01:28, 17.74s/it]

...

 71%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 5/7 [01:29<00:35, 17.83s/it]

 channel:   5 304


 86%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 6/7 [01:46<00:17, 17.83s/it]

 channel:   6 335


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:04<00:00, 17.82s/it]


In [260]:
fig,ax = plt.subplots(2,3,figsize = (8,5),sharex=True,sharey=True)
ax = ax.flatten()

im0 = ax[0].imshow((imgs[0,:,:]),cmap= plt.get_cmap('sdoaia94'),norm = LogNorm())
im0.set_clim(0.0001,np.percentile(imgs[0,:,:],95))
im1 = ax[1].imshow((imgs[1,:,:]),cmap= plt.get_cmap('sdoaia131'),norm = LogNorm())
im1.set_clim(0.0001,np.percentile(imgs[1,:,:],95))
im2 = ax[2].imshow((imgs[2,:,:]),cmap= plt.get_cmap('sdoaia171'),norm = LogNorm())
im2.set_clim(0.0001,np.percentile(imgs[2,:,:],95))
im3 = ax[3].imshow((imgs[3,:,:]),cmap= plt.get_cmap('sdoaia193'),norm = LogNorm())
im3.set_clim(0.0001,np.percentile(imgs[3,:,:],95))
im4 = ax[4].imshow((imgs[4,:,:]),cmap= plt.get_cmap('sdoaia211'),norm = LogNorm())
im4.set_clim(0.0001,np.percentile(imgs[4,:,:],95))
## skip 304 
## 
im5 = ax[5].imshow((imgs[6,:,:]),cmap= plt.get_cmap('sdoaia335'),norm = LogNorm())    
im5.set_clim(0.0001,np.percentile(imgs[6,:,:],95))

fig.tight_layout(pad=0.01)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
plt.figure()
ax = plt.gca()
for n in range(7): 
    iint1 = rgi((zza[:,0],yya[0,:]),imgs[n,:,:],method = 'linear',fill_value = 0.,bounds_error = False)
    ax.plot(np.linspace(1.02,2.,200),iint1(zy),'.-',label = chnames[n].decode())
ax.set_yscale('log')
ax.legend()

NameError: name 'plt' is not defined